Задание: бинарная классификация - определить в аудио говорят Barbie или Puppy??? Данных мало -> необходима аугментация

In [2]:
!pip install audiomentations


  Attempting uninstall: soxr

    Found existing installation: soxr 1.0.0

    Uninstalling soxr-1.0.0:

      Successfully uninstalled soxr-1.0.0

   ---------------------------------------- 0/5 [soxr]
   -------- ------------------------------- 1/5 [python-stretch]
   ------------------------ --------------- 3/5 [numpy-minmax]
   -------------------------------- ------- 4/5 [audiomentations]
   -------------------------------- ------- 4/5 [audiomentations]
   -------------------------------- ------- 4/5 [audiomentations]
   -------------------------------- ------- 4/5 [audiomentations]
   -------------------------------- ------- 4/5 [audiomentations]
   -------------------------------- ------- 4/5 [audiomentations]
   ---------------------------------------- 5/5 [audiomentations]



In [4]:
!pip install kagglehub


   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   ---------------------------------------- 0/2 [kagglesdk]
   -------------------- ------------------- 1/2 [kagglehub]
   -------------------- ------------------- 1/2 [kagglehub]
   -------------------- ------------------- 1/2 [kagglehub]
   -------------------- ------------------- 1/2 [kagglehub]
   ------------------------------------

In [5]:
import os
import numpy as np
import librosa
import pandas as pd

In [6]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
import audiomentations as A

import pickle # сериализация/десериализация
import kagglehub

In [9]:
path = kagglehub.dataset_download("levshaazz/audio-binary-classification-barbie-vs-puppy")

100%|█████████████████████████████████████████████████████████████████████████████| 15.6M/15.6M [00:02<00:00, 6.03MB/s]

Extracting files...


In [10]:
barbie_folder = path + r"\\barbie_vs_puppy\\barbie"
puppy_folder = path + r"\\barbie_vs_puppy\\puppy"

In [11]:
barbie_files = [os.path.join(barbie_folder, f) for f in os.listdir(barbie_folder) if f.endswith('.wav')]
puppy_files = [os.path.join(puppy_folder, f) for f in os.listdir(puppy_folder) if f.endswith('.wav')]

In [12]:
print(f"Barbie: {len(barbie_files)}")
print(f"Puppy: {len(puppy_files)}")

Barbie: 50
Puppy: 48


In [13]:
augment = A.Compose([
    A.AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.005, p=0.3),
    A.TimeStretch(min_rate=0.9, max_rate=1.1, p=0.3),
    A.PitchShift(min_semitones=-2, max_semitones=2, p=0.3),
])

In [14]:
# Получаем фичи для обучения классификатора
def extract_features(file_path, apply_augmentation=False):
    audio, sr = librosa.load(file_path, sr=22050, duration=3.0)
    
    if apply_augmentation:
        audio = augment(samples=audio, sample_rate=sr)
    
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    
    mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=40)
    mel_mean = np.mean(mel, axis=1)
    
    features = np.concatenate([mfcc_mean, mel_mean])
    
    return features

In [15]:
X_list = []
y_list = []
file_paths = []

In [16]:
# Загружаем Barbie аудио
for file in barbie_files:
    features = extract_features(file, apply_augmentation=False)
    X_list.append(features)
    y_list.append(0)
    file_paths.append(file)

In [17]:
# Загружаем Puppy аудио
for file in puppy_files:
    features = extract_features(file, apply_augmentation=False)
    X_list.append(features)
    y_list.append(1)
    file_paths.append(file)

In [18]:
X = np.array(X_list)
y = np.array(y_list)
file_paths = np.array(file_paths)

In [19]:
# Split (train, val, test)
indices = np.arange(len(X))

idx_temp, idx_test = train_test_split(indices, test_size=0.2, random_state=42, stratify=y)
idx_train, idx_val = train_test_split(idx_temp, test_size=0.25, random_state=42, stratify=y[idx_temp])

X_train = X[idx_train]
y_train = y[idx_train]
files_train = file_paths[idx_train]

X_val = X[idx_val]
y_val = y[idx_val]

X_test = X[idx_test]
y_test = y[idx_test]

print(f"Train: {len(X_train)}")
print(f"Val: {len(X_val)}")
print(f"Test: {len(X_test)}")

Train: 58
Val: 20
Test: 20


In [20]:
# Увеличиваем обучающую выборку
X_train_aug = []
y_train_aug = []

for file_path in files_train:
    for _ in range(5):
        features = extract_features(file_path, apply_augmentation=True)
        X_train_aug.append(features)
        idx = np.where(file_paths == file_path)[0][0]
        y_train_aug.append(y[idx])

X_train_combined = np.vstack([X_train, np.array(X_train_aug)])
y_train_combined = np.concatenate([y_train, np.array(y_train_aug)])

print(f"Train после аугментации: {len(X_train_combined)}")

Train после аугментации: 348


In [21]:
# Нормализация
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [38]:
def build_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [39]:
model = build_model(X_train_scaled.shape[1])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001
)

history = model.fit(
    X_train_scaled, y_train_combined,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=8,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5057 - loss: 0.8018 - val_accuracy: 0.4500 - val_loss: 0.7014 - learning_rate: 0.0010
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6034 - loss: 0.6898 - val_accuracy: 0.6000 - val_loss: 0.6435 - learning_rate: 0.0010
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6379 - loss: 0.6006 - val_accuracy: 0.7500 - val_loss: 0.6155 - learning_rate: 0.0010
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7155 - loss: 0.5083 - val_accuracy: 0.7500 - val_loss: 0.5914 - learning_rate: 0.0010
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7299 - loss: 0.5152 - val_accuracy: 0.7500 - val_loss: 0.5705 - learning_rate: 0.0010
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7644 - loss: 0.4370 - val_accuracy: 0.6500 - val_loss: 0.5547 - learning_rate: 0.0010
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7902 - loss: 0.4325 - val_ac

ИТОГО: на train - accuracy= 0.9368, на val - 0,7346.

In [24]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train_combined, verbose=0)
val_loss, val_acc = model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy:   {val_acc:.4f}")
print(f"Test Accuracy:  {test_acc:.4f}")

Train Accuracy: 0.9052
Val Accuracy:   0.7000
Test Accuracy:  0.8000


In [25]:
def predict_audio(file_path, model, scaler):
    features = extract_features(file_path, apply_augmentation=False)
    features = features.reshape(1, -1)
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled, verbose=0)[0][0]
    
    if prediction > 0.5:
        return "I hear Puppy", prediction
    else:
        return "I hear Barbie", 1 - prediction

In [27]:
# Проверка предсказания на файле из исходного датасета
test_file = barbie_files[0]
label, confidence = predict_audio(test_file, model, scaler)
print(f"Файл: {os.path.basename(test_file)}")
print(f"Предсказание: {label}")
print(f"Уверенность: {confidence:.2%}")

Файл: barbie_0.wav
Предсказание: I hear Barbie
Уверенность: 77.14%


In [40]:
file_path = '../data/puppy.wav'

label, confidence = predict_audio(file_path, model, scaler)

print(f"Файл: {os.path.basename(file_path)}")
print(f"Предсказание: {label}")
print(f"Уверенность: {confidence:.2%}")

Файл: puppy.wav
Предсказание: I hear Puppy
Уверенность: 75.93%


In [41]:
file_path = '../data/barbie.wav'

label, confidence = predict_audio(file_path, model, scaler)

print(f"Файл: {os.path.basename(file_path)}")
print(f"Предсказание: {label}")
print(f"Уверенность: {confidence:.2%}")

Файл: barbie.wav
Предсказание: I hear Barbie
Уверенность: 53.17%


Уверенность в ответах не самая высокая (53,17 - почти угадывание), думаю, 300 сэмплов - всё ещё очень маленький датасет для обучения, нужно сделать ещё больше тренировочных данных для обучения модели.